In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import random
import seaborn as sns
pd.options.display.max_rows=300
pd.set_option('display.max_columns', None)

In [2]:
df_type = pd.read_csv("../../01.MIMICIV_SAKI_trajCluster/df_mixAK_fea4_C3.csv")
df_type_filt = df_type.loc[:,["stay_id","groupHPD"]]
df_type_filt = df_type_filt.drop_duplicates()
all_lst = df_type_filt.stay_id.unique().tolist()
print("纳入患者数量：",len(all_lst))

df_type_filt_count = pd.DataFrame(df_type_filt["groupHPD"].value_counts()).reset_index()
df_type_filt_count.columns = ["groupHPD","count_all"]
df_type_filt_count

纳入患者数量： 4713


,groupHPD,count_all
0,2,3055
1,1,1298
2,3,360


In [3]:
df_time = pd.read_csv("/public/hanl/jupyter_dir/kidney_sepsis_penotype_v3/00.data_mimic/disease_definition/AKI/sk_event_time.csv")
df_time = df_time[["stay_id","sepsis_onset", "saki_onset"]]
df_time = pd.merge(df_type_filt, df_time, how="inner", on="stay_id")
df_time["sepsis_onset"] = pd.to_datetime(df_time["sepsis_onset"])
df_time["saki_onset"] = pd.to_datetime(df_time["saki_onset"])
df_time.head(2)

,stay_id,groupHPD,sepsis_onset,saki_onset
0,30003598,1,2189-04-14 21:00:00,2189-04-15 03:04:00
1,30004144,2,2126-04-05 00:00:00,2126-04-05 06:00:00


# 利尿剂

In [ ]:
# 220987	D-Mannitol (C6H14O6)
# 220988	Mannitol 5%
# 220989	Mannitol 10%
# 220990	Mannitol 15%
# 220991	Mannitol 20%
# 220992	Mannitol 25%

# 227531	Mannitol
# 221794	Furosemide (Lasix)
# 228340	Furosemide (Lasix) 250/50
# 229639	Bumetanide (Bumex)

df_di = pd.read_csv("../../00.data_mimic/treatment/diuretic.csv")
df_di = df_di[(df_di.amountuom != "dose") & (df_di.itemid != 227531)]
df_di = df_di[['stay_id', 'starttime', 'endtime','amount', 'amountuom',"itemid","ordercategoryname"]]
df_di["starttime"] = pd.to_datetime(df_di["starttime"] )
df_di["endtime"] = pd.to_datetime(df_di["endtime"])
df_di.loc[df_di["itemid"]==229639,"amount"] *= 80
df_di.loc[df_di["ordercategoryname"]=='01-Drips',"amount"]  *= 2
df_di.loc[df_di["ordercategoryname"]=='05-Med Bolus',"amount"]  *= 1
df_di = df_di[['stay_id', 'starttime', 'endtime','amount']] # mg单位,且全部转为mg PO furosemide
df_di.loc[df_di["amount"]>df_di["amount"].quantile(0.999),"amount"] = 0 # df_di["amount"].quantile(0.99) = 774

import datetime
def dateHourRange(beginDateHour, endDateHour):
    beginDateHour= beginDateHour.strftime('%Y-%m-%d %H:%M:%S')
    endDateHour= endDateHour.strftime('%Y-%m-%d %H:%M:%S')
    dhours = []
    dhour = datetime.datetime.strptime(beginDateHour, "%Y-%m-%d %H:%M:%S")
    date = beginDateHour[:]
    while date <= endDateHour:
        dhours.append(date)
        dhour = dhour + datetime.timedelta(hours=1)
        date = dhour.strftime("%Y-%m-%d %H:%M:%S")
    return dhours

tem_di = df_di
tem_di["time_lst"] = tem_di.apply(lambda x: dateHourRange(x['starttime'],x['endtime']),axis=1)
tem_di["time_num"] = tem_di["time_lst"].map(lambda x: len(x))
tem_di["amount_hourly"] =  tem_di["amount"]/tem_di["time_num"] 

tem_di['idx'] = range(len(tem_di))
tem_add = tem_di.set_index('idx').time_lst.apply(pd.Series).stack().reset_index(level=0).rename(columns={0:'charttime'})
tem_di_hourly = pd.merge(tem_di, tem_add,how="inner", on="idx")
tem_di_hourly = tem_di_hourly.loc[:,['stay_id', 'charttime',  'amount_hourly']] # 'starttime', 'endtime', 'amount',
tem_di_hourly = tem_di_hourly.rename(columns={"amount_hourly":"amount"})
tem_di_hourly["charttime"] = pd.to_datetime(tem_di_hourly["charttime"] )
tem_di_hourly.head(2)

In [ ]:
df_di = tem_di_hourly
df_di = df_di[df_di["stay_id"].isin(all_lst)]
df_time_cut = df_time[['stay_id', 'saki_onset']]
df_di = pd.merge(df_di,df_time_cut, how="right",on="stay_id")
df_di = df_di[df_di["charttime"] >= df_di["saki_onset"]]
df_di = df_di.sort_values(["stay_id","charttime"])
df_di.columns = ["stay_id","diuretic_time","diuretic_amout","saki_onset"]
df_di = df_di.drop(["saki_onset"],axis=1)
df_di.head(1)

# 尿量

In [ ]:
import datetime
def dateHourRange_v2(beginDateHour, endDateHour):
    beginDateHour= beginDateHour.strftime('%Y-%m-%d %H:%M:%S')
    endDateHour= endDateHour.strftime('%Y-%m-%d %H:%M:%S')
    dhours = []
    dhour = datetime.datetime.strptime(endDateHour, "%Y-%m-%d %H:%M:%S")
    date = endDateHour[:]
    while date > beginDateHour:
        dhours.append(date)
        dhour = dhour - datetime.timedelta(hours=1)
        date = dhour.strftime("%Y-%m-%d %H:%M:%S")
    return dhours

In [ ]:
df_uo = pd.read_csv("../../00.data_mimic/feature_data/sk_icu_feature.csv",usecols=["stay_id","charttime","urineoutput"])
df_uo = df_uo[df_uo["stay_id"].isin(all_lst)]
df_uo["charttime"] = pd.to_datetime(df_uo["charttime"])
df_uo = df_uo[~df_uo.urineoutput.isnull()]
tmp = df_uo
tmp['last_charttime'] = tmp.groupby('stay_id')['charttime'].shift(1)
tmp["last_charttime"].fillna(tmp["charttime"]- datetime.timedelta(hours=1), inplace=True)


tmp["time_lst"] = tmp.apply(lambda x: dateHourRange_v2(x['last_charttime'],x['charttime']),axis=1)
tmp["time_num"] = tmp["time_lst"].map(lambda x: len(x))
tmp["urineoutput_hourly"] =  tmp["urineoutput"]/tmp["time_num"] 

tmp['idx'] = range(len(tmp))
tem_add = tmp.set_index('idx').time_lst.apply(pd.Series).stack().reset_index(level=0).rename(columns={0:'uo_charttime_hourly'})
tmp_hourly = pd.merge(tmp, tem_add,how="inner", on="idx")
tmp_hourly["uo_charttime_hourly"] = pd.to_datetime(tmp_hourly["uo_charttime_hourly"] )
df_uo_hourly = tmp_hourly[["stay_id","uo_charttime_hourly","urineoutput_hourly"]]
df_uo_hourly = df_uo_hourly.sort_values(["stay_id","uo_charttime_hourly"])
df_uo_hourly.head()

# 定义利尿反应性

In [ ]:
df_merge = pd.merge(df_di, df_uo_hourly, how="left", on="stay_id")
print(df_merge.shape)
df_merge=df_merge[(df_merge["uo_charttime_hourly"]<=df_merge["diuretic_time"]+datetime.timedelta(hours=24)) & (df_merge["uo_charttime_hourly"]>=df_merge["diuretic_time"]-datetime.timedelta(hours=24))]
print(df_merge.shape)

df_merge["uo_charttime_hourly_b"] = df_merge["uo_charttime_hourly"] - datetime.timedelta(hours=1)
time_win = 2
df_merge["diuretic_time_b"] = df_merge["diuretic_time"] - datetime.timedelta(hours=time_win)
df_merge["diuretic_time_a"] = df_merge["diuretic_time"] + datetime.timedelta(hours=time_win)
df_merge.head()

In [ ]:
# 取利尿剂使用前2h的尿量
df_bt = df_merge 
df_bt['intersection'] = df_bt.apply(lambda row: pd.date_range(max(row['uo_charttime_hourly_b'], row['diuretic_time_b']), min(row['uo_charttime_hourly'], row['diuretic_time']), freq='S').tolist(), axis=1)
df_bt['intersection_duration'] = df_bt['intersection'].apply(lambda x: pd.Timedelta(seconds=len(x)))
df_bt['intersection_duration'] = round(pd.to_timedelta(df_bt['intersection_duration']).dt.total_seconds() / 60)
df_bt["urineoutput_before_useDiu"] = df_bt["urineoutput_hourly"] * df_bt["intersection_duration"] / 60
df_bt = df_bt.groupby(["stay_id","diuretic_time","diuretic_amout"]).agg({"urineoutput_before_useDiu":"sum"}).reset_index()
df_bt = pd.merge(df_di,df_bt,how="left",on=["stay_id","diuretic_time","diuretic_amout"])
df_bt = df_bt.fillna(0)
df_bt = df_bt.drop_duplicates()
#df_bt.to_csv("df_bt.csv",index=False)
df_bt.head()

In [ ]:
df_bt.shape,df_bt[df_bt["urineoutput_before_useDiu"]<200].shape

In [ ]:
# 取利尿剂使用后2h的尿量
df_at = df_merge 
df_at['intersection'] = df_at.apply(lambda row: pd.date_range(max(row['uo_charttime_hourly_b'], row['diuretic_time']), min(row['uo_charttime_hourly'], row['diuretic_time_a']), freq='S').tolist(), axis=1)
df_at['intersection_duration'] = df_at['intersection'].apply(lambda x: pd.Timedelta(seconds=len(x)))
df_at['intersection_duration'] = round(pd.to_timedelta(df_at['intersection_duration']).dt.total_seconds() / 60)
df_at["urineoutput_after_useDiu"] = df_at["urineoutput_hourly"] * df_at["intersection_duration"] / 60
df_at = df_at.groupby(["stay_id","diuretic_time","diuretic_amout"]).agg({"urineoutput_after_useDiu":"sum"}).reset_index()
df_at = pd.merge(df_di,df_at,how="left",on=["stay_id","diuretic_time","diuretic_amout"])
df_at = df_at.fillna(0)
df_at = df_at.drop_duplicates()
#df_at.to_csv("df_at.csv",index=False)
df_at.head()

In [ ]:
df_at.shape,df_at[df_at["urineoutput_after_useDiu"]<200].shape

In [4]:
df_bt = pd.read_csv("df_bt.csv")
df_at = pd.read_csv("df_at.csv")
change_percent = 0.1
df_bt_at = pd.merge(df_bt, df_at, how="inner",on=["stay_id","diuretic_time","diuretic_amout"])
df_bt_at.loc[(df_bt_at["urineoutput_after_useDiu"] <= df_bt_at["urineoutput_before_useDiu"] + df_bt_at["urineoutput_before_useDiu"]*change_percent) | (df_bt_at["urineoutput_after_useDiu"] <=200),"one_label_diu_res"] = "Non-responsive"
df_bt_at.loc[((df_bt_at["urineoutput_after_useDiu"] > 200) & (df_bt_at["urineoutput_before_useDiu"]>200)) | ((df_bt_at["urineoutput_after_useDiu"] > df_bt_at["urineoutput_before_useDiu"] + df_bt_at["urineoutput_before_useDiu"]*change_percent) & (df_bt_at["urineoutput_after_useDiu"] >200)),"one_label_diu_res"] = "responsive"

df_bt_at.to_csv("tmp_df_diuretic_responsitive.csv",index=False)

In [5]:
df_stat = df_bt_at.groupby("stay_id").agg({'one_label_diu_res': lambda x: list(x)}).reset_index()

def process_list(lst):
    if len(lst) > 2:
        counts = {}
        for item in lst:
            counts[item] = counts.get(item, 0) + 1
        max_count = max(counts.values())
        most_common = [k for k, v in counts.items() if v == max_count]
        if len(most_common) > 1:
            return most_common[1]
        else:
            return most_common[0]
    elif len(lst) == 2:
        return lst[-1]
    else:
        return lst[0]

df_stat['label_diu_res'] = df_stat['one_label_diu_res'].apply(process_list)
df_stat.head()

,stay_id,one_label_diu_res,label_diu_res
0,30003598,"[responsive, responsive]",responsive
1,30004144,[responsive],responsive
2,30005085,"[responsive, responsive, responsive]",responsive
3,30006983,"[responsive, responsive, responsive, responsiv...",responsive
4,30008148,"[responsive, responsive, responsive, Non-respo...",responsive


In [6]:
tmp = pd.merge(df_bt_at,df_stat[['stay_id', 'label_diu_res']],how="inner",on="stay_id")
df_bt_at = tmp[["stay_id","label_diu_res"]].drop_duplicates()
df_bt_at.label_diu_res.value_counts()

responsive        2433
Non-responsive     510
Name: label_diu_res, dtype: int64

In [7]:
df_sur = pd.read_csv("../../01.MIMICIV_SAKI_trajCluster/sk_survival.csv")
df_sur = df_sur[['stay_id','groupHPD', 'mortality_28d', 'survival_28day']]
df_bt_at = pd.merge(df_bt_at,df_sur, how="right",on="stay_id")
df_bt_at["label_diu_res"] = df_bt_at["label_diu_res"].fillna("No diuretic")
df_bt_at.head(1)

,stay_id,label_diu_res,groupHPD,mortality_28d,survival_28day
0,37510196.0,No diuretic,1,1.0,7.0


In [8]:
df_bt_at.to_csv("df_diuretic_FST.csv",index=False)

# 合并输出

In [22]:
df_bt_at = pd.read_csv("df_diuretic_FST.csv")
result = df_bt_at.groupby(["groupHPD", "label_diu_res"])["stay_id"].agg(["count"]).reset_index()
df_count_all = pd.DataFrame(df_bt_at.groupby("groupHPD")["stay_id"].agg("count")).reset_index()
df_count_all.columns = ["groupHPD","groupHPD_count"]

result = pd.merge(result,df_count_all,how="inner",on="groupHPD")
result["count_percentage"] = result["count"] / result["groupHPD_count"] *100
result

,groupHPD,label_diu_res,count,groupHPD_count,count_percentage
0,1,No diuretic,396,1298,30.508475
1,1,Non-responsive,184,1298,14.175655
2,1,responsive,718,1298,55.315871
3,2,No diuretic,1211,3055,39.639935
4,2,Non-responsive,211,3055,6.906710
5,2,responsive,1633,3055,53.453355
6,3,No diuretic,163,360,45.277778
7,3,Non-responsive,115,360,31.944444
8,3,responsive,82,360,22.777778


In [30]:
df_fea = pd.read_csv("./mimic_diuretic_survival.csv")
df_fea = df_fea[df_fea["time"]==1]
#'crea_divide_basecrea',
df_fea = df_fea[['stay_id',  'creatinine', 'urineoutput','baseline_Scr',
                 'colloid_bolus','sofa_norenal',"Congestive_heart_failure",
                  'weight', 'age', 'gender', 'height', 
               
       'respiration_sofa', 'cardiovascular_sofa', 'aniongap', 'pt', 'sbp',
    'coagulation_sofa', 'cum_fluidbance', 'is_diu',
       'baseexcess',   'hemoglobin',
         'liver_sofa',
       'furosemide_amount', 'fluidbance', 'gcs_verbal', 'gcs', 'mbp',
       'potassium', 'spo2', 'platelet', 'renal_sofa', 'gcs_eyes',
       'bicarbonate', 'sodium', 'temperature', 'gcs_motor', 'po2', 
       'ptt', 'glucose', 'pco2', 'chloride', 'rbc', 'wbc',
       'ph', 'sofa', 'mchc',  'mcv', 'hematocrit',
       'first_use_amount', 'cns_sofa',  'time',
       'heart_rate', 'fio2',  'resp_rate',  
       'furosemide_amount_kg', 'dbp', 'mch', 'is_diu_byid', 'lactate', 'rdw',
       'calcium', 'first_use_time' 
              ]]
df_bt_at_ = pd.merge(df_bt_at,df_fea, how="right",on="stay_id")
df_bt_at_.to_csv("df_diuretic_responsitive.csv",index=False)
df_bt_at_.label_diu_res.value_counts()

responsive        2433
No diuretic       1770
Non-responsive     510
Name: label_diu_res, dtype: int64

In [32]:
df_bt_at_.head()

,stay_id,label_diu_res,groupHPD,mortality_28d,survival_28day,creatinine,urineoutput,baseline_Scr,colloid_bolus,sofa_norenal,Congestive_heart_failure,weight,age,gender,height,respiration_sofa,cardiovascular_sofa,aniongap,pt,sbp,coagulation_sofa,cum_fluidbance,is_diu,baseexcess,hemoglobin,liver_sofa,furosemide_amount,fluidbance,gcs_verbal,gcs,mbp,potassium,spo2,platelet,renal_sofa,gcs_eyes,bicarbonate,sodium,temperature,gcs_motor,po2,ptt,glucose,pco2,chloride,rbc,wbc,ph,sofa,mchc,mcv,hematocrit,first_use_amount,cns_sofa,time,heart_rate,fio2,resp_rate,furosemide_amount_kg,dbp,mch,is_diu_byid,lactate,rdw,calcium,first_use_time
0,30003598.0,responsive,1,0.0,15.540278,1.6375,1450.0,1.1,4584.131181,3.666667,0.0,136.0,64.0,0,165.0,2.000000,1.000,14.875,19.8375,111.723214,0.000000,3726.72,0.0,-6.500000,9.950000,0.0,0.000000,3726.72,1.25,14.750000,69.688988,5.0375,99.720238,406.50,1.00,2.875000,18.500,132.75,37.078750,5.875,167.937500,32.8250,150.500000,39.875,104.375,3.450,16.2000,7.294375,4.666667,33.250,87.00,30.000000,20.000001,0.666667,1,91.927083,50.0,22.880952,0.00,50.379464,28.8500,1,2.600000,17.800,8.1250,4.0
1,30004144.0,responsive,2,0.0,6.251389,0.8000,802.0,0.7,1904.762432,4.291667,0.0,80.0,82.0,1,168.0,1.583333,1.000,15.500,13.5500,132.833333,0.000000,1102.76,0.0,-0.250000,11.775000,0.0,0.000000,1102.76,2.00,14.708333,73.694444,4.0750,98.458333,225.75,0.00,2.916667,23.500,135.50,37.063750,5.500,166.000000,32.3500,122.500000,35.750,100.500,3.725,12.6250,7.420000,4.291667,33.700,93.50,34.041667,10.000001,1.708333,1,87.035256,42.5,20.697917,0.00,50.387500,31.5500,1,1.250000,13.125,8.5750,3.0
2,30005085.0,responsive,2,0.0,4.475000,0.8000,1982.0,0.7,386.272663,3.541667,0.0,101.4,60.0,1,183.0,1.333333,1.000,10.000,12.8000,128.125000,0.958333,-1595.73,1.0,-0.800000,10.200000,0.0,40.000003,-1595.73,4.75,14.750000,76.270833,4.3250,94.625000,134.00,0.00,4.000000,26.000,138.25,36.515000,6.000,179.800000,29.9000,102.319444,44.400,106.250,3.260,9.3000,7.356000,3.541667,33.300,94.00,30.700000,40.000003,0.250000,1,86.875000,50.0,14.541667,0.39,57.875000,31.4000,1,1.800000,13.800,8.7125,1.0
3,30006983.0,responsive,2,0.0,30.000000,1.2375,885.0,0.8,4649.010999,10.875000,0.0,69.4,78.0,1,168.0,3.000000,2.875,17.875,20.3875,106.600000,2.000000,3764.01,0.0,-4.791667,9.141667,3.0,0.000000,3764.01,0.00,15.000000,74.722917,3.5750,97.589583,93.00,0.75,2.125000,18.625,140.50,36.874167,5.000,114.541667,38.0875,97.651786,34.500,107.500,2.775,19.8875,7.358750,11.625000,33.525,99.75,27.375000,20.000001,0.000000,1,92.112500,75.0,20.276515,0.00,58.102083,33.3625,1,5.675000,17.675,8.3250,5.0
4,30008148.0,responsive,2,0.0,12.375000,0.6750,4272.0,0.7,1211.864056,1.125000,0.0,106.9,61.0,1,179.0,0.166667,0.000,11.500,12.5500,135.751984,0.000000,-2843.99,1.0,0.500000,10.600000,0.0,40.000003,-2843.99,5.00,14.937500,91.430556,4.2750,96.583333,176.50,0.00,3.937500,28.500,137.00,37.097500,6.000,93.000000,31.4750,191.700000,42.500,101.000,3.605,12.0500,7.407500,1.125000,33.350,88.50,31.750000,20.000001,0.958333,1,79.013889,40.0,17.000000,0.37,69.680556,29.4000,1,1.420833,13.100,8.6000,-1.0
